In [4]:
import sys
sys.path.append("..")
import pickle

In [5]:
from games.abstract_poker import AbstractPokerRootChanceGameState
from games.algorithms import ChanceSamplingCFR, VanillaCFR
from common.constants import card_dealing, results_map
import numpy as np
import pandas as pd

from games.competition_abstract_poker import CompetitionAbstractPoker
from copy import deepcopy

In [6]:
# Hyperparameters
_NUM_TRAIN_ITERS = 1_000
_COMPETITION_ROUNDS = 1_000
_RUN_SINGLE_EXAMPLE = True

# Try single example

In [18]:
root = AbstractPokerRootChanceGameState(max_hand_strength = 5, 
                                    max_turns = 2,
                                    ante = 0.1,
                                    report = False)
bot = ChanceSamplingCFR(root)
bot.run(iterations = 100)
bot.compute_nash_equilibrium()

max_strength = bot.root.max_hand_strength

for i in range(max_strength):
    p1 = chance_sampling_cfr.nash_equilibrium[f'.{i+1}.']['BET']
    p2 = chance_sampling_cfr.nash_equilibrium[f'.{i+1}.CHECK_INITIATE']['BET']
    print(f"Bet probability as 1st player for hand strength {i+1}/{max_strength} is {p1}")
    print(f"Bet probability as 2nd player for hand strength {i+1}/{max_strength} is {p2}")

Bet probability as 1st player for hand strength 1/5 is 0.0931013431013431
Bet probability as 2nd player for hand strength 1/5 is 0.03571428571428571
Bet probability as 1st player for hand strength 2/5 is 0.2433451827367531
Bet probability as 2nd player for hand strength 2/5 is 0.05555555555555555
Bet probability as 1st player for hand strength 3/5 is 0.19643287539166063
Bet probability as 2nd player for hand strength 3/5 is 0.1387600732800005
Bet probability as 1st player for hand strength 4/5 is 0.18665169200457501
Bet probability as 2nd player for hand strength 4/5 is 0.23765204699105924
Bet probability as 1st player for hand strength 5/5 is 0.5613322544798371
Bet probability as 2nd player for hand strength 5/5 is 0.06818181818181818


# Granularity competition

In [23]:
def train_bots_fixed_depth(granularity, depth = 2, report = True):
    bots = []

    for g in granularity:
        root = AbstractPokerRootChanceGameState(max_hand_strength = g, 
                                            max_turns = depth,
                                            report = False)
        chance_sampling_cfr = ChanceSamplingCFR(root)
        chance_sampling_cfr.run(iterations = _NUM_TRAIN_ITERS)
        chance_sampling_cfr.compute_nash_equilibrium()
        bots.append(chance_sampling_cfr)
        
    if report:
        print("------------------------------------------------------------")
        for b in bots:
            print(f"Value of game: {b.value_of_the_game():.5f}")
        
    return bots

In [24]:
def train_bots_fixed_granularity(depth, granularity = 5, report = True):
    bots = []

    for d in depth:
        root = AbstractPokerRootChanceGameState(max_hand_strength = granularity, 
                                            max_turns = d,
                                            report = False)
        chance_sampling_cfr = ChanceSamplingCFR(root)
        chance_sampling_cfr.run(iterations = _NUM_TRAIN_ITERS)
        chance_sampling_cfr.compute_nash_equilibrium()
        bots.append(chance_sampling_cfr)
        
    if report:
        print("------------------------------------------------------------")
        for b in bots:
            print(f"Value of game: {b.value_of_the_game():.5f}")
        
    return bots

## Train static bots

In [25]:
ante_list = [np.round(0.2*i,1) for i in range(1,11)]
granularity_bots = train_bots_fixed_depth(granularity = [3,5,7,10], depth = 2)

------------------------------------------------------------
Value of game: -0.03539
Value of game: -0.07604
Value of game: -0.04785
Value of game: -0.04393


In [28]:
depth_bots = train_bots_fixed_granularity(granularity = 5, depth = [1,2,3])

------------------------------------------------------------
Value of game: -0.03842
Value of game: -0.02570
Value of game: -0.06576


# Get strategies

In [49]:
def get_aggression(bots):
    results = []
    
    for b in bots:
        g = b.root.max_hand_strength
        d = b.root.max_turns
        
        for i in range(1,g+1):
            p1 = b.nash_equilibrium[f'.{i}.']['BET']
            p2 = b.nash_equilibrium[f'.{i}.CHECK_INITIATE']['BET']
            results.append([g,d,i,p1,p2])
            
    return results

In [50]:
all_bots = granularity_bots + depth_bots

In [51]:
aggressiveness = get_aggression(all_bots)

# Export results 

In [52]:
results_df = pd.DataFrame(aggressiveness, columns = ['granularity', 'depth', 'hand_strength',
                                                            'intial_bet', 'bet_after_check'])

In [53]:
results_df.head(3)

,granularity,depth,hand_strength,intial_bet,bet_after_check
0,3,2,1,0.171929,0.639959
1,3,2,2,0.137909,0.092438
2,3,2,3,0.350222,0.986831


In [54]:
results_df.to_csv("output/agression_by_granularity_and_depth.csv", index = False)